In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

from scipy.stats import norm


In [ ]:
def plot(freq):
    x = np.linspace(0, 2*np.pi)
    y = np.sin(x * freq)
    plt.plot(x, y, '--k')
    plt.ylim((-1, 1))
    plt.show()

widgets.interact(plot, freq=widgets.FloatSlider(value=2, min=1, max=5, step=0.1))

In [ ]:
n_samp = 64
x = np.linspace(norm.ppf(0.01), norm.ppf(0.99), n_samp)
pdf = norm.pdf(x)
cdf = norm.cdf(x)

def plot(ii):
    plt.fill_between(x[:ii], np.zeros(x[:ii].shape), norm.pdf(x[:ii]), color='C0', label='pdf')
    plt.plot(x[:ii], norm.cdf(x[:ii]), color='C1', label='cdf')
#     plt.plot([x[:ii], x[:ii]], [0, 1])

    plt.ylim((0, 1))
    plt.xlim((x[0], x[-1]))
    plt.legend()

plt.figure(num=0, clear=True)
widgets.interact(plot, ii=widgets.IntSlider(value=0, min=0, max=n_samp, step=3))


In [ ]:
pfa = 1e-3
n_bkg = 64
n_pt = 2**16
samps = np.arange(n_pt)

kernel = np.concatenate(
    (
        np.ones(n_bkg//2),
        np.zeros(1),
        np.ones(n_bkg//2)
    )
)
kernel  # Average
threshold = n_bkg * (pfa**(-1/n_bkg) - 1)

# sig = 1/np.sqrt(2) * (np.random.randn(n_pt) + 1.0j * np.random.randn(n_pt))
sig = np.random.randn(n_pt)
sig_pwr = np.abs(sig)**2
bkg_est = 1/n_bkg * np.convolve(sig_pwr, kernel, mode='same')
detections = sig_pwr > bkg_est * threshold 

fig, axs = plt.subplots(
    nrows=1,
    ncols=2,
    clear=True,
    num=2
)

axs[0].plot(samps, sig_pwr)
axs[0].plot(samps, bkg_est*threshold, '--')
axs[1].hist(sig_pwr, bins=64)
axs[1].plot([threshold, threshold], axs[1].get_ylim())

print(np.sum(detections)/n_pt)

In [ ]:
n_pt = 2**11
n_bkg = 32  # Number of cells used in background power estimate
n_guard = 5  # Number of guard cells

# sig = np.random.randn(n_pt)
sig = 1/np.sqrt(2) * (np.random.randn(n_pt) + 1.0j * np.random.randn(n_pt))
sig_pwr = np.abs(sig)**2

def plot_data(n):
    pfa = 10**(-n)
    samps = np.arange(n_pt)

    kernel = np.concatenate(
        (np.ones(n_bkg//2), np.zeros(n_guard), np.ones(n_bkg//2))
    )
    threshold = n_bkg * (pfa**(-1/n_bkg) - 1)

    # Get average background power estimate
    bkg_est = 1/n_bkg * np.convolve(sig_pwr, kernel, mode='same')
    detections = sig_pwr > bkg_est * threshold
    
    id_detect = np.argwhere(detections).ravel()

    plt.plot(samps, sig_pwr)
    plt.plot(samps, bkg_est*threshold, '--')
    plt.plot(samps[id_detect], sig_pwr[id_detect], 'rx')
    meas_pfa = np.sum(detections) / n_pt
    plt.title(f'{meas_pfa=:.2e}\n{pfa=:.2e}')

def plot_hist(n):
    pfa = 10**(-n)
    threshold = n_bkg * (pfa**(-1/n_bkg) - 1)

    plt.hist(sig_pwr, bins=64)
    plt.plot([threshold, threshold], plt.gca().get_ylim())

    
w = widgets.FloatSlider(value=1, min=0, max=6, step=0.1)

plt.figure(clear=True, num=3)
widgets.interact(plot_data, n=w)

plt.figure(clear=True, num=4)
widgets.interact(plot_hist, n=w)


# Test
Theory behind notebook.  Probability of false alarm $P_{fa}$ is defined as
$$
P_{fa} = \frac{d}{N}
$$

In [ ]:
10**(-2)